Osnabrück University - Machine Learning (Summer Term 2016) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 10

## Introduction

This week's sheet should be solved and handed in before the end of **Sunday, June 28, 2016**. If you need help (and Google and other resources were not enough), feel free to contact your groups' designated tutor or whomever of us you run into first. Please upload your results to your group's Stud.IP folder.

## Assignment 1: Ultimate Dinosaur 3000 Xtrem!  [10 Points]

In this assignment your task will be to use unsupervised learning methods and create the greatest dinosaur-labyrinth simulation that the world has ever seen. The group with the highest score reached by your AI will receive a special treat from your trainer.

In [ ]:
def generate_labyrinth:
    '''
    Generates a maze following Prim's algorithm. The labyrinth is stored in a matrix.
    1's encode walls and 0's a passage.
    
    -Start with a grid full of walls.
    -Pick a cell, mark it as part of the maze. Add the walls of the cell to the wall list.
    -While there are walls in the list:
        -Pick a random wall from the list. We now have two cases: either there exists 
         exactly one unvisited cell on one of the two sides of the chosen wall, or there does not. If it is the former:
        -Make the wall a passage and mark the unvisited cell as part of the maze.
        -Add the neighboring walls of the cell to the wall list.
        -Remove the wall from the list.
    '''
    
    
    

## Assignment 2:  [x Points]
